# Get Jurisdiction Demographics

* About ACS: https://www.census.gov/data/developers/data-sets/acs-5year.html
* Census API documentation: https://www.census.gov/content/dam/Census/data/developers/api-user-guide/api-guide.pdf
* Uses [`census`](https://pypi.org/project/census/) Python library to get ACS data (median household income, % people of color, and % cost-burdened households) for relevant geographies. May need to run `pip install census` to install the library prior to running this notebook.
* Detailed profiles: https://api.census.gov/data/2020/acs/acs5/profile/variables.html
* Regular tables: https://api.census.gov/data/2020/acs/acs5/variables.html

In [1]:
import pandas as pd
from census import Census

In [18]:
# Initialize Census with API key
c = Census('a62cc2eb31421c58e5f3e445fd8bf689e0bd155b')

# Define geographic level
county_geo = {
    'for': 'county:*',
    'in': 'state:30'
}

county_subdivision_geo = {
    'for': 'county subdivision:*',
    'in': 'state:30'
}

year = 2020 # ACS release year

### Get median household income per jurisdiction

In [50]:
# Get data for all towns (county subdvisions) in MT (state code 30)
# Get data for all counties and county subdivisions in MT
county_income_raw = c.acs5.get(
    ['NAME', 'B19013_001E'], 
    county_geo,
    year=year
)

# Create DataFrame and rename columns
county_income = pd.DataFrame(
    county_income_raw
).rename(
    columns={
        'B19013_001E': 'income'
    }
).filter(['NAME', 'income', 'county', 'county subdivision'])

In [51]:
# Get data for all county subdivisions in MT
income_raw_cs = c.acs5.get(
    ['NAME', 'B19013_001E'], 
    county_subdivision_geo, 
    year=year
)

# Create DataFrame and rename columns
income_cs = pd.DataFrame(
    income_raw_cs
).rename(
    columns={
        'B19013_001E': 'income'
    }
).filter(['NAME', 'income', 'county', 'county subdivision'])

# Append county subdivision data to income DataFrame
income = pd.concat([county_income, pd.DataFrame(income_cs)])
income.head()

,NAME,income,county,county subdivision
0,"Big Horn County, Montana",48273.0,003,NaN
1,"Blaine County, Montana",45361.0,005,NaN
2,"Carbon County, Montana",61209.0,009,NaN
3,"Carter County, Montana",42300.0,011,NaN
4,"Chouteau County, Montana",45707.0,015,NaN


In [29]:
income.shape

(112, 3)

### Calculate % BIPOC per jurisdiction

In [36]:
# Get data for all counties in MT
bipoc_raw_county = c.acs5dp.get(
    ['NAME', 'DP05_0077PE'], 
    county_geo,
    year=year
)

# Create DataFrame and rename columns
bipoc_county = pd.DataFrame(
    bipoc_raw_county
).assign(
    bipoc=lambda df_: round(100 - df_.DP05_0077PE, 1)
).filter(['NAME', 'bipoc', 'county', 'county subdivision'])

# Get data for all county subdivisions in MT
bipoc_raw_cs = c.acs5dp.get(
    ['NAME', 'DP05_0077PE'], 
    county_subdivision_geo, 
    year=year
)

# Create DataFrame and rename columns
bipoc_cs = pd.DataFrame(
    bipoc_raw_cs
).assign(
    bipoc=lambda df_: round(100 - df_.DP05_0077PE, 1)
).filter(['NAME', 'bipoc', 'county', 'county subdivision'])

# Append county subdivision data to bipoc DataFrame
bipoc = pd.concat([bipoc_cs, pd.DataFrame(bipoc_county)])
bipoc.head()

,NAME,bipoc,county,county subdivision
0,"Northern Cheyenne Reservation CCD, Big Horn Co...",97.4,003,92436
1,"Townsend East CCD, Broadwater County, Montana",4.8,007,93570
2,"Joliet CCD, Carbon County, Montana",5.5,009,91827
3,"Alzada-Boyes CCD, Carter County, Montana",0.0,011,90052
4,"Eden-Stockett CCD, Cascade County, Montana",4.2,013,91029


### Calculate % cost-burdened households per jurisdiction

In [38]:
# Get data for all counties in MT
costburdened_raw_county = c.acs5dp.get(
    ['NAME',
     # owners, with mortgage:
     'DP04_0110E', 'DP04_0114E', 'DP04_0115E',
     # owners, without mortgage:
     'DP04_0117E', 'DP04_0123E', 'DP04_0124E',
     # renters:
     'DP04_0136E', 'DP04_0141E', 'DP04_0142E'
    ],
    county_geo,
    year=year
)

# Create DataFrame and rename columns
costburdened_county = pd.DataFrame(
    costburdened_raw_county
).assign(
    num=lambda df_: df_.DP04_0114E + df_.DP04_0115E \
        + df_.DP04_0123E + df_.DP04_0124E + df_.DP04_0141E + df_.DP04_0142E,
    denom=lambda df_: df_.DP04_0110E + df_.DP04_0117E + df_.DP04_0136E,
    burdened=lambda df_: round(df_.num / df_.denom*100, 1)
).filter(['NAME', 'burdened', 'county', 'county subdivision'])

# Get data for all county subdivisions in MT
costburdened_raw_cs = c.acs5dp.get(
    ['NAME',
     # owners, with mortgage:
     'DP04_0110E', 'DP04_0114E', 'DP04_0115E',
     # owners, without mortgage:
     'DP04_0117E', 'DP04_0123E', 'DP04_0124E',
     # renters:
     'DP04_0136E', 'DP04_0141E', 'DP04_0142E'
    ], 
    county_subdivision_geo, 
    year=year
)

# Create DataFrame and rename columns
costburdened_cs = pd.DataFrame(
    costburdened_raw_cs
).assign(
    num=lambda df_: df_.DP04_0114E + df_.DP04_0115E \
        + df_.DP04_0123E + df_.DP04_0124E + df_.DP04_0141E + df_.DP04_0142E,
    denom=lambda df_: df_.DP04_0110E + df_.DP04_0117E + df_.DP04_0136E,
    burdened=lambda df_: round(df_.num / df_.denom*100, 1)
).filter(['NAME', 'burdened', 'county', 'county subdivision'])

# Append county subdivision data to costburdened DataFrame
costburdened = pd.concat([costburdened_county, pd.DataFrame(costburdened_cs)])

costburdened.head()

,NAME,burdened,county,county subdivision
0,"Big Horn County, Montana",20.7,003,NaN
1,"Blaine County, Montana",26.4,005,NaN
2,"Carbon County, Montana",24.9,009,NaN
3,"Carter County, Montana",27.2,011,NaN
4,"Chouteau County, Montana",17.4,015,NaN


### Combine three variables into a single dataframe, and save

In [42]:
demographics = (income
                .merge(bipoc, on='NAME')
                .merge(costburdened, on='NAME')
                .assign(
                    NAME=lambda df_: df_.NAME.apply(lambda x: x.split(' town,')[0])
                )
               )


In [52]:
income['county'] = income['county'].astype(str).fillna('')
income['county subdivision'] = income['county subdivision'].astype(str).fillna('')
costburdened['county'] = costburdened['county'].astype(str).fillna('')
costburdened['county subdivision'] = costburdened['county subdivision'].astype(str).fillna('')
bipoc['county'] = bipoc['county'].astype(str).fillna('')
bipoc['county subdivision'] = bipoc['county subdivision'].astype(str).fillna('')

In [53]:
# Concatenate the 'county' and 'county_subdivision' columns
income['region'] = income.apply(lambda row: row['county'] + row['county subdivision'], axis=1)
costburdened['region'] = costburdened.apply(lambda row: row['county'] + row['county subdivision'], axis=1)
bipoc['region'] = bipoc.apply(lambda row: row['county'] + row['county subdivision'], axis=1)

# Join the dataframes on the concatenated 'region' column
demographics = income.merge(bipoc, on='region')
demographics = demographics.merge(costburdened, on='region')

# Drop the 'region' column
demographics = demographics.drop(columns=['region'])

In [55]:
# Drop the specified columns
demographics = demographics.drop(columns=['NAME_x', 'county_x', 'county subdivision_x', 'NAME_y',
                                           'county_y', 'county subdivision_y'])


In [57]:
# Reset the index
demographics = demographics.reset_index()

# Save to file
with open(r'web-map\data\demographics.js', 'w') as f:
    f.write(
        'const demographics = ' +\
        str(demographics #[~demographics.NAME.str.startswith('County')]
         .set_index('NAME')
         .to_dict('index')
        )
    )